In [1]:
import os
import time
import google.generativeai as genai
from youtube_transcript_api import YouTubeTranscriptApi
import yt_dlp
from dotenv import load_dotenv

# --- CONFIGURATION ---
load_dotenv()

API_KEY = os.getenv('GOOGLE_API_KEY')
if not API_KEY:
    raise ValueError("No API Key found. Please set GOOGLE_API_KEY in .env file")

genai.configure(api_key=API_KEY)

# Use 2.5 Flash for speed and context window
model = genai.GenerativeModel('gemini-2.5-flash')

def get_video_id(url):
    """Extracts video ID from URL."""
    if "youtu.be" in url:
        return url.split("/")[-1].split("?")[0]
    if "v=" in url:
        return url.split("v=")[1].split("&")[0]
    return None

def format_timestamp(seconds):
    """Converts seconds (float) to HH:MM:SS or MM:SS string."""
    seconds = int(seconds)
    m, s = divmod(seconds, 60)
    h, m = divmod(m, 60)
    if h > 0:
        return f"{h:02d}:{m:02d}:{s:02d}"
    return f"{m:02d}:{s:02d}"

def download_audio(url, output_filename="audio.mp3"):
    """Downloads audio using yt-dlp (Plan B)."""
    print(f"⬇️  Plan B: Downloading audio for {url}...")
    
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'outtmpl': output_filename.replace('.mp3', ''),
        'quiet': True,
        'no_warnings': True,
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        print("✅ Audio downloaded successfully.")
        return output_filename
    except Exception as e:
        print(f"❌ Error downloading audio: {e}")
        return None

def analyze_transcript_data(transcript_object):
    """Formats the FetchedTranscript object and sends to Gemini."""
    print("🧠 Formatting transcript data...")
    
    full_text = ""

    # FIX: Access the 'snippets' attribute from your object
    # The log shows: FetchedTranscript(snippets=[...])
    if hasattr(transcript_object, 'snippets'):
        entries = transcript_object.snippets
    else:
        # Fallback if it somehow comes as a raw list
        entries = transcript_object

    for entry in entries:
        # FIX: Access attributes via dot notation (.text) not dictionary keys ['text']
        # The log shows: FetchedTranscriptSnippet(text="...", start=0.08, ...)
        text = getattr(entry, 'text', '')
        start = getattr(entry, 'start', 0.0)
        
        time_str = format_timestamp(start)
        full_text += f"[{time_str}] {text}\n"

    print(f"📤 Sending {len(full_text)} characters to Gemini...")

    prompt = (
        "You are an expert video analyst. I will provide a transcript with timestamps. "
        "Your task is to create a structured summary.\n\n"
        "Output Format:\n"
        "## 📝 Executive Summary\n"
        "(A concise paragraph summarizing the main idea)\n\n"
        "## ⏱️ Key Topics & Timestamps\n"
        "(A bulleted list of key moments with their exact timestamps found in the text)\n"
        "- **[MM:SS]** Topic Description\n\n"
        f"TRANSCRIPT DATA:\n{full_text}"
    )
    
    response = model.generate_content(prompt)
    return response.text

def analyze_audio(audio_path):
    """Uploads audio to Gemini for native listening."""
    print("🧠 Uploading audio to Gemini (may take a moment)...")
    
    audio_file = genai.upload_file(path=audio_path)
    
    while audio_file.state.name == "PROCESSING":
        print('.', end='', flush=True)
        time.sleep(1)
        audio_file = genai.get_file(audio_file.name)

    print("\n✅ Audio processed. Generating summary...")
    
    prompt = (
        "Listen to this audio clip from a YouTube video. "
        "Create a structured summary with estimated timestamps.\n\n"
        "Output Format:\n"
        "## 📝 Executive Summary\n"
        "(A concise paragraph)\n\n"
        "## ⏱️ Key Topics (Approximate Times)\n"
        "- **[MM:SS]** Topic Description"
    )
    
    response = model.generate_content([prompt, audio_file])
    return response.text

def run_agent(youtube_url):
    video_id = get_video_id(youtube_url)
    if not video_id:
        print("❌ Invalid YouTube URL")
        return

    print(f"🚀 Starting Agent for Video ID: {video_id}")

    # --- PLAN A: Fetch Transcript (Using your specific library usage) ---
    try:
        print("Attempting Plan A: Fetching transcript...")
        
        # FIX: Instantiating class as per your request
        ytt_api = YouTubeTranscriptApi()
        
        # This returns a FetchedTranscript object
        transcript_data = ytt_api.fetch(video_id)
        
        summary = analyze_transcript_data(transcript_data)
        print("\n" + "="*50)
        print(summary)
        print("="*50)
        return

    except Exception as e:
        print(f"⚠️ Plan A failed. Error: {e}")

    # --- PLAN B: Download Audio (Approximate Timestamps) ---
    try:
        audio_file = download_audio(youtube_url)
        if audio_file and os.path.exists(audio_file):
            summary = analyze_audio(audio_file)
            print("\n" + "="*50)
            print(summary)
            print("="*50)
            
            os.remove(audio_file)
            print("🧹 Cleanup: Temporary audio file deleted.")
        else:
            print("❌ Plan B failed: Could not download audio.")

    except Exception as e:
        print(f"❌ Plan B failed with error: {e}")

# --- RUN IT ---
if __name__ == "__main__":
    link = 'https://www.youtube.com/watch?v=GxDTx1bx-x0'
    run_agent(link)

/home/valka/repo/.scrape/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_179978/1322785040.py:3: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  import google.generativeai as genai


🚀 Starting Agent for Video ID: GxDTx1bx-x0
Attempting Plan A: Fetching transcript...
🧠 Formatting transcript data...
📤 Sending 10035 characters to Gemini...

## 📝 Executive Summary
This video outlines a proactive strategy for generating income by identifying and solving problems for local businesses, particularly restaurants, using Google Maps data and AI analysis. The process involves finding businesses with low ratings, extracting and analyzing their customer reviews using AI tools, and then crafting professional summaries of identified issues and potential solutions. The initial analysis and outreach via cold email are offered for free to impress potential clients. Monetization occurs by offering solutions like connecting businesses with third-party service providers (e.g., for pest control), implementing tools like NFC review cards, or developing AI-powered automated review response systems and SEO optimization for their Google Maps presence. The core idea emphasizes active problem

In [3]:
print("GOOGLE_API_KEY =", os.getenv("GOOGLE_API_KEY"))
print("CWD =", os.getcwd())

GOOGLE_API_KEY = YOUR_API_KEY_HERE
CWD = /home/valka/repo/youtube/summarization


In [4]:
print(os.getcwd())

/home/valka/repo/youtube/summarization


In [14]:
link = 'https://www.youtube.com/watch?v=GxDTx1bx-x0'


video_id = get_video_id(link)

ytt_api = YouTubeTranscriptApi()
transcript_data = ytt_api.fetch(video_id)     
transcript_data

FetchedTranscript(snippets=[FetchedTranscriptSnippet(text="In this video, we'll be focusing on", start=0.08, duration=4.32), FetchedTranscriptSnippet(text='finding clients, getting Google Maps', start=2.24, duration=5.599), FetchedTranscriptSnippet(text='reviews, AI analysis, creating the', start=4.4, duration=6.56), FetchedTranscriptSnippet(text="asset, and reaching out by email. Let's", start=7.839, duration=4.88), FetchedTranscriptSnippet(text='dive into it. The first step is going to', start=10.96, duration=4.24), FetchedTranscriptSnippet(text='be finding clients. We can search in', start=12.719, duration=4.32), FetchedTranscriptSnippet(text='Google Maps by restaurants by typing', start=15.2, duration=4.72), FetchedTranscriptSnippet(text='restaurants here. And we can search for', start=17.039, duration=6.08), FetchedTranscriptSnippet(text='specific restaurants that are star below', start=19.92, duration=6.32), FetchedTranscriptSnippet(text='four or three stars. What we need to do',

In [3]:
import youtube_transcript_api
print(f"Library is loading from: {youtube_transcript_api.__file__}")

Library is loading from: /home/valka/repo/.scrape/lib/python3.11/site-packages/youtube_transcript_api/__init__.py


In [5]:
import google.generativeai as genai
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv('GOOGLE_API_KEY')
genai.configure(api_key=api_key)

print("🔍 Checking available models for your API Key...")
try:
    for m in genai.list_models():
        if 'generateContent' in m.supported_generation_methods:
            print(f" - Found: {m.name}")
except Exception as e:
    print(f"❌ Error connecting to Google: {e}")

🔍 Checking available models for your API Key...
 - Found: models/gemini-2.5-flash
 - Found: models/gemini-2.5-pro
 - Found: models/gemini-2.0-flash-exp
 - Found: models/gemini-2.0-flash
 - Found: models/gemini-2.0-flash-001
 - Found: models/gemini-2.0-flash-lite-001
 - Found: models/gemini-2.0-flash-lite
 - Found: models/gemini-2.0-flash-lite-preview-02-05
 - Found: models/gemini-2.0-flash-lite-preview
 - Found: models/gemini-exp-1206
 - Found: models/gemini-2.5-flash-preview-tts
 - Found: models/gemini-2.5-pro-preview-tts
 - Found: models/gemma-3-1b-it
 - Found: models/gemma-3-4b-it
 - Found: models/gemma-3-12b-it
 - Found: models/gemma-3-27b-it
 - Found: models/gemma-3n-e4b-it
 - Found: models/gemma-3n-e2b-it
 - Found: models/gemini-flash-latest
 - Found: models/gemini-flash-lite-latest
 - Found: models/gemini-pro-latest
 - Found: models/gemini-2.5-flash-lite
 - Found: models/gemini-2.5-flash-image
 - Found: models/gemini-2.5-flash-preview-09-2025
 - Found: models/gemini-2.5-flash-li